# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json


# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = "./WeatherPy.csv"
vacation_df = pd.read_csv(weather_data)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hirara,24.8000,125.2833,80.28,89,97,8.28,JP,1666383577
1,Saint Paul Harbor,57.7900,-152.4072,49.51,46,0,17.27,US,1666383578
2,Walvis Bay,-22.9575,14.5053,61.75,82,1,9.22,NaN,1666383578
3,Turukhansk,65.8167,87.9833,33.91,98,100,18.41,RU,1666383578
4,Nikolskoye,59.7035,30.7861,42.69,93,100,3.27,RU,1666383579


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
coordinates = [
    (38.8976763, -77.0365298),
    (47.6062095, -122.3320708),
    (40.7127753, -74.0059728),
    (30.267153, -97.7430608),
    (44.977753, -93.2650108),     
]
markers = gmaps.marker_layer(coordinates)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [18]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [13]:
figure_layout = {
'width': '400px',
'height': '300px',
'border': '1px solid black',
'padding': '1px',
'margin': '0 auto 0 auto'
}
gmaps.figure(layout=figure_layout)

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

In [19]:
#Use the Lat and Lng as locations and Humidity as the weight.
#store lat / long data
locations = vacation_df[["Lat", "Lng"]].astype(float)
humidity = vacation_df["Humidity"].astype(float)

figure_layout = {
'width': '400px',
'height': '300px',
'border': '1px solid black',
'padding': '1px',
'margin': '0 auto 0 auto'
}
gmaps.figure(layout=figure_layout)

#create a heat map by location
fig = gmaps.figure(map_type="TERRAIN")
fig = gmaps.figure(layout=figure_layout)


heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

# fig.add_layer(gmaps.heatmap_layer(locations, weights=humidity, 
#                                  dissipating=False, max_intensity=100,
#                                  point_radius = 1))

fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Drop rows that have null values
new_vacation_data = vacation_df.dropna()
#set the temerature range of the ideal location
clean_vacation_data = new_vacation_data[(new_vacation_data['Max Temp'] >= 65) 
                                        & (new_vacation_data['Max Temp'] < 80)
                                        & (new_vacation_data['Wind Speed'] < 10)
                                        & (new_vacation_data['Cloudiness'] == 0)]
clean_vacation_data.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
79,Tura,25.5198,90.2201,67.64,77,0,5.84,IN,1666383602
98,Chhāpar,27.8167,74.4000,77.04,30,0,6.04,IN,1666383611
126,Rome,34.2570,-85.1647,73.90,28,0,3.44,US,1666383606
128,Mount Isa,-20.7333,139.5000,74.30,32,0,8.23,AU,1666383625
139,Gachsaran,30.3586,50.7981,71.69,53,0,0.00,IR,1666383629


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ""
hotel_df

,Hotel Name


In [9]:
radius = 5000
query = 'hotel'
lat = "34.2570"
lng = "-85.1647"

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "query": query,
    "location": f"{lat},{lng}",
    "radius": radius,
    "rankby": "distance",
    "key": g_key,
}
# run a request using our params dictionary
response = requests.get(base_url, params=params)
response = response.json()


In [10]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
